In [13]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os
import pandas as pd

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display


In [14]:
# Model Initialization
model_name = "movenet_lightning"
# module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
input_size = 192
model = module.signatures['serving_default']


In [16]:
# Choose image file and preprocess

image_path = "../clean_data/TRAIN/downdog/00000128.jpg"
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)


In [17]:
# Detect landmarks on image
model = module.signatures["serving_default"]
input_image = tf.cast(input_image, dtype=tf.int32)
input_image = input_image[..., :3]
outputs = model(input_image)

# 1D array of xyz coordinates. X1, Y1, Z1, X2, ... , Yn, Zn format
xyz = outputs["output_0"].numpy().reshape(51).tolist()


In [18]:
# Helper method for drawing landmarks
def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot


# Extract landmarks from all files in a directory

In [114]:
# Get list of image file names in given directory
dir_path_train_dd = "../clean_data/TRAIN/warrior2/warrior2-right/"
file_names = []
for entry in os.listdir(dir_path_train_dd):
    file_names.append(entry)


In [115]:
# Create DF of xyz values
df = []
xyz = []
for file in file_names:
    dir_path_train_dd = f"../clean_data/TRAIN/warrior2/warrior2-right/{file}"
    if not file.endswith((".jpg", ".png")):
        pass
    else:
        print(dir_path_train_dd)
        image = tf.io.read_file(dir_path_train_dd)
        image = tf.image.decode_jpeg(image)
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)
        input_image = input_image[..., :3]

        model = module.signatures["serving_default"]
        input_image = tf.cast(input_image, dtype=tf.int32)
        outputs = model(input_image)
        to_df = outputs["output_0"].numpy().reshape(51).tolist()
        to_df.insert(0, dir_path_train_dd)
        df.append(to_df)

        # Append the unflattened data for angle analysis
        # xyz.append(outputs["output_0"].numpy()[0][0])


../clean_data/TRAIN/warrior2/warrior2-right/00000428.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000158.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000171.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000173.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000167.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000205.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000204.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000238.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000358.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000162.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000215.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000217.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000259.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000270.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000266.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000299.jpg
../clean_data/TRAIN/warrior2/warrior2-right/00000303.jpg
../clean_data/TRAIN/warrior2/wa

In [116]:
# Run this
a = None


In [117]:
# Change to DataFrame
a = pd.DataFrame(df)

# Add extra y1 and y2 labels
a["y_main"] =  "warrior2"
a["y_sub"] = "warrior2-right"


In [118]:
final_df = pd.concat([final_df, a], ignore_index=True)


In [119]:
final_df.shape


(535, 54)

In [120]:
final_df.tail()


,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,y_main,y_sub
530,../clean_data/TRAIN/warrior2/warrior2-right/00...,0.392098,0.709279,0.621784,0.382245,0.710954,0.620449,0.382438,0.701716,0.603388,...,0.567370,0.777585,0.719250,0.806177,0.518862,0.722182,0.468859,0.644198,warrior2,warrior2-right
531,../clean_data/TRAIN/warrior2/warrior2-right/00...,0.298483,0.580572,0.741461,0.288884,0.572129,0.586976,0.291347,0.564433,0.527163,...,0.363522,0.748143,0.738138,0.683170,0.656149,0.747080,0.231454,0.905582,warrior2,warrior2-right
532,../clean_data/TRAIN/warrior2/warrior2-right/00...,0.279541,0.373823,0.633221,0.265342,0.372076,0.683806,0.265546,0.361493,0.740337,...,0.222063,0.769277,0.737224,0.485421,0.810906,0.732894,0.122998,0.809422,warrior2,warrior2-right
533,../clean_data/TRAIN/warrior2/warrior2-right/00...,0.271259,0.554901,0.806328,0.256909,0.551478,0.886625,0.260788,0.528610,0.847673,...,0.335752,0.515160,0.756966,0.718744,0.906823,0.766991,0.197909,0.752597,warrior2,warrior2-right
534,../clean_data/TRAIN/warrior2/warrior2-right/00...,0.344049,0.584744,0.577472,0.330424,0.598922,0.561891,0.330703,0.572625,0.639093,...,0.446958,0.712662,0.751754,0.747287,0.745660,0.746810,0.364119,0.807862,warrior2,warrior2-right


In [121]:
final_df.to_csv("landmark_all_raw.csv", index=False)
